In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K

In [2]:
config = tf.ConfigProto() # 定义TensorFlow配置
config.gpu_options.allow_growth = True # 配置GPU动态分配，按需增长 
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
K.set_session(sess)  # set this TensorFlow session as the default session for Keras

In [3]:
def load_data(path="./data/"):
    features = np.load(path+'fea.npy')
    adj = np.load(path+'adj.npy')
    props = np.load(path+'prop.npy')
    return features, adj, props

In [4]:
features, adj, props = load_data()

In [5]:
print("features shape: ", features.shape, "\n", "adj shape: ", adj.shape, "\n", "props shape: ", props.shape)

features shape:  (1000, 50, 58) 
 adj shape:  (1000, 50, 50) 
 props shape:  (1000,)


In [6]:
class GCN(keras.layers.Layer):
    def __init__(self, output_dim):
        self.output_dim = output_dim
        super(GCN, self).__init__()

    def build(self, input_shape):
        self.built = True
        shape = tf.TensorShape((input_shape[0][2], self.output_dim)).as_list()  # [58 , 32]

        with K.name_scope('Parameters'):
            with K.name_scope('Weight_' + str(shape[1])):
                self.kernel = self.add_weight(name='conv_weight',
                                              shape=shape,
                                              initializer='glorot_uniform',
                                              trainable=True)

            with K.name_scope('Bias_' + str(shape[1])):
                self.bias = self.add_weight(name='conv_bias',
                                            shape=(shape[1],),
                                            initializer='glorot_uniform',
                                            trainable=True)
    def call(self, inputs):
        X, A = inputs[0], inputs[1]
        with K.name_scope('Prop.Rule_' + str(X.get_shape()[2])):
            b = tf.reshape(tf.tile(self.bias, [X.get_shape()[1]]), [X.get_shape()[1], self.output_dim])
            output = tf.einsum('ijk,kl->ijl', X, self.kernel) + b
            return output

    def compute_output_shape(self, input_shape):
        return input_shape[0][0], input_shape[0][1], self.output_dim

In [7]:
class G2N(keras.layers.Layer):
    def __init__(self, output_dim):
        self.output_dim = output_dim
        super(G2N, self).__init__()

    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[2], self.output_dim)).as_list()

        with K.name_scope('Parameters'):
            with K.name_scope('Weight' + str(shape[1])):
                self.kernel = self.add_weight(name='permu_weight',
                                              shape=shape,
                                              initializer='glorot_uniform',
                                              trainable=True)
        self.built = True

    def call(self, X):
        with K.name_scope('ATOMWISE'):
            Z = tf.einsum('ijk,kl->ijl', X, self.kernel)
            Z = K.relu(Z)
            Z = K.sigmoid(K.sum(Z, axis=1))
            return Z

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.output_dim

In [8]:
def build_model():
    X = keras.layers.Input(shape=(50, 58))
    A = keras.layers.Input(shape=(50, 50))

    with K.name_scope('graph_conv-1.{}'.format(X.get_shape()[2])):
        gcn1 = GCN(32)
        x = keras.layers.ReLU()(gcn1([X, A]))
    with K.name_scope('graph_conv-2.{}'.format(x.get_shape()[2])):
        gcn2 = GCN(32)
        x = keras.layers.ReLU()(gcn2([x, A]))
    with K.name_scope('Perm-Invariance'):
        g2n = G2N(64)
        x = keras.layers.ReLU()(g2n(x))
    with K.name_scope('Latent_Space-64'):
        x = keras.layers.Dense(64, activation="relu")(x)
    with K.name_scope('Latent_Space-64'):
        x = keras.layers.Dense(64, activation="relu")(x)
    with K.name_scope('Output'):
        output = keras.layers.Dense(1)(x)

    model = keras.models.Model(inputs=[X, A], outputs=output)
    return model


In [9]:
model = build_model()


In [10]:
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50, 58)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50, 50)       0                                            
__________________________________________________________________________________________________
gcn (GCN)                       (None, 50, 32)       1888        input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 50, 32)       0           gcn[0][0]                        
__________

In [11]:
lr = 0.001
tensorboard = keras.callbacks.TensorBoard(log_dir="./summaries/ADAM-0.001",histogram_freq=10,batch_size=100, 
                          write_graph=True, write_grads=True)
optimizer = keras.optimizers.Adam(lr)


In [12]:
model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mean_absolute_error', 'mean_squared_error'])


In [13]:
history = model.fit(x=[features, adj], y=props, batch_size=100, 
                    epochs=1000, validation_split=0.1, verbose=0,
                    callbacks=[tensorboard])

In [18]:
K.clear_session()

In [14]:
## Test with SGD optimizer

In [19]:
lr = 0.001
model = build_model()
tensorboard = keras.callbacks.TensorBoard(log_dir="./summaries/SGD-0.001",histogram_freq=10,batch_size=100, 
                          write_graph=True, write_grads=True)
optimizer = keras.optimizers.SGD(lr)
model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mean_absolute_error', 'mean_squared_error'])

In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50, 58)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50, 50)       0                                            
__________________________________________________________________________________________________
gcn (GCN)                       (None, 50, 32)       1888        input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 50, 32)       0           gcn[0][0]                        
__________

In [21]:
history = model.fit(x=[features, adj], y=props, batch_size=100, 
                    epochs=1000, validation_split=0.1, verbose=0,
                    callbacks=[tensorboard])

In [22]:
K.clear_session()
lr = 0.001
model = build_model()
tensorboard = keras.callbacks.TensorBoard(log_dir="./summaries/RMS-0.001",histogram_freq=10,batch_size=100, 
                          write_graph=True, write_grads=True)
optimizer = keras.optimizers.RMSprop(lr)
model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mean_absolute_error', 'mean_squared_error'])

history = model.fit(x=[features, adj], y=props, batch_size=100, 
                    epochs=1000, validation_split=0.1, verbose=0,
                    callbacks=[tensorboard])